In [1]:
!pip install -q segmentation-models
!pip install -q albumentations
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.9 MB/s eta 0:00:00


2026-02-08 12:50:41.518868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770555041.696137      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770555041.750561      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770555042.204756      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770555042.204790      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770555042.204793      55 computation_placer.cc:177] computation placer alr

Segmentation Models: using `tf.keras` framework.


In [5]:

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

import cv2
import glob
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import albumentations as A
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, callbacks, backend as K

# --- 1. CONFIGURATION ---
DATASET_ROOT = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle"
TRAIN_IMG_DIR = os.path.join(DATASET_ROOT, "train_images")
TRAIN_MASK_DIR = os.path.join(DATASET_ROOT, "train_masks")
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "test_images_padded")

# Correct Resolution & Backbone
IMG_H = 544 
IMG_W = 960 
BATCH_SIZE = 4 
EPOCHS = 7    
LR = 1e-4
N_FOLDS = 5     

BACKBONE = 'seresnext50' 
preprocess_input = sm.get_preprocessing(BACKBONE)

# --- 2. AUGMENTATION (Optimized for Terrain) ---
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    # Vertical Flip removed (Terrain is never upside down)
    A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=15, shift_limit=0.1, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RGBShift(p=0.3), # Crucial for lighting changes
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
])

# --- 3. DATA LOADER ---
def process_path(file_name, augment=False):
    file_name = file_name.decode()
    img_path = os.path.join(TRAIN_IMG_DIR, file_name)
    mask_path = os.path.join(TRAIN_MASK_DIR, file_name)
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if augment:
        aug = train_transform(image=img, mask=mask)
        img, mask = aug['image'], aug['mask']
        
    img = cv2.resize(img, (IMG_W, IMG_H))
    mask = cv2.resize(mask, (IMG_W, IMG_H), interpolation=cv2.INTER_NEAREST)
    
    img = preprocess_input(img).astype(np.float32)
    mask = np.isin(mask, [27, 39]).astype(np.float32)
    
    return img, mask[..., np.newaxis]

def tf_process_dataset(file_paths, is_train=False):
    ds = tf.data.Dataset.from_tensor_slices(file_paths)
    if is_train:
        ds = ds.shuffle(len(file_paths))
        ds = ds.map(lambda x: tf.numpy_function(process_path, [x, True], [tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(lambda x: tf.numpy_function(process_path, [x, False], [tf.float32, tf.float32]), num_parallel_calls=tf.data.AUTOTUNE)
    
    ds = ds.map(lambda x, y: (tf.ensure_shape(x, [IMG_H, IMG_W, 3]), tf.ensure_shape(y, [IMG_H, IMG_W, 1])))
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

# --- 4. MODEL & LOSS ---
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = sm.losses.DiceLoss()(y_true, y_pred)
    return bce + dice

def build_model():
    model = sm.Unet(
        BACKBONE, 
        classes=1, 
        activation='sigmoid', 
        encoder_weights='imagenet',
        decoder_block_type='transpose'
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LR),
        loss=bce_dice_loss,
        metrics=[sm.metrics.IOUScore(threshold=0.5)]
    )
    return model

# --- 5. TRAINING LOOP (K-FOLD) ---
all_files = np.array(sorted(os.listdir(TRAIN_IMG_DIR)))
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
best_thresholds = []

print(f"🚀 Starting {N_FOLDS}-Fold Cross-Validation with {BACKBONE}...")

for fold, (train_idx, val_idx) in enumerate(kf.split(all_files)):
    print(f"\n--- FOLD {fold+1}/{N_FOLDS} ---")
    
    train_ds = tf_process_dataset(all_files[train_idx], is_train=True)
    val_ds = tf_process_dataset(all_files[val_idx], is_train=False)
    
    K.clear_session()
    model = build_model()
    
    checkpoint_name = f'model_fold_{fold+1}.keras'
    checkpoint = callbacks.ModelCheckpoint(checkpoint_name, save_best_only=True, monitor='val_iou_score', mode='max', verbose=0)
    lr_schedule = callbacks.LearningRateScheduler(lambda epoch: LR * 0.5 * (1 + np.cos(np.pi * epoch / EPOCHS)))
    
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint, lr_schedule],
        verbose=1
    )
    
    # Validation Threshold Tuning
    model.load_weights(checkpoint_name)
    val_preds = model.predict(val_ds, verbose=0)
    val_masks = np.vstack([mask.numpy() for _, mask in val_ds])
    
    best_iou, best_th = 0, 0.5
    for th in np.arange(0.3, 0.7, 0.05):
        score = sm.metrics.IOUScore(threshold=th)(val_masks, val_preds).numpy()
        if score > best_iou: best_iou, best_th = score, th
            
    best_thresholds.append(best_th)
    print(f"✅ Fold {fold+1} Best Threshold: {best_th:.2f} | IoU: {best_iou:.4f}")
    
    del model, train_ds, val_ds, val_preds, val_masks
    gc.collect()

avg_threshold = np.mean(best_thresholds)
print(f"\n🏆 Average Best Threshold: {avg_threshold:.3f}")

/tmp/ipykernel_55/3610870800.py:40: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),


🚀 Starting 5-Fold Cross-Validation with seresnext50...

--- FOLD 1/5 ---
Epoch 1/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 738s 749ms/step - iou_score: 0.7665 - loss: 0.5916 - val_iou_score: 0.8559 - val_loss: 0.3242 - learning_rate: 1.0000e-04
Epoch 2/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 342s 538ms/step - iou_score: 0.8513 - loss: 0.3270 - val_iou_score: 0.8655 - val_loss: 0.2991 - learning_rate: 9.5048e-05
Epoch 3/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 342s 538ms/step - iou_score: 0.8625 - loss: 0.2956 - val_iou_score: 0.8709 - val_loss: 0.2810 - learning_rate: 8.1174e-05
Epoch 4/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 342s 539ms/step - iou_score: 0.8688 - loss: 0.2806 - val_iou_score: 0.8748 - val_loss: 0.2713 - learning_rate: 6.1126e-05
Epoch 5/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 342s 538ms/step - iou_score: 0.8715 - loss: 0.2722 - val_iou_score: 0.8778 - val_loss: 0.2662 - learning_rate: 3.8874e-05
Epoch 6/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 341s 537ms/step - iou_score: 0.8729 - loss: 0.2679 - val_iou_score: 0.8785 - val_lo

2026-02-08 14:40:39.791571: E external/local_xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module a_inference_one_step_on_data_883320__.89303] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2026-02-08 14:40:53.721590: E external/local_xla/xla/service/slow_operation_alarm.cc:140] The operation took 2m13.930150911s

********************************
[Compiling module a_inference_one_step_on_data_883320__.89303] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


635/635 ━━━━━━━━━━━━━━━━━━━━ 740s 738ms/step - iou_score: 0.7684 - loss: 0.5545 - val_iou_score: 0.8534 - val_loss: 0.3286 - learning_rate: 1.0000e-04
Epoch 2/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 336s 529ms/step - iou_score: 0.8539 - loss: 0.3168 - val_iou_score: 0.8675 - val_loss: 0.2945 - learning_rate: 9.5048e-05
Epoch 3/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 337s 530ms/step - iou_score: 0.8609 - loss: 0.2958 - val_iou_score: 0.8694 - val_loss: 0.2833 - learning_rate: 8.1174e-05
Epoch 4/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 337s 530ms/step - iou_score: 0.8690 - loss: 0.2792 - val_iou_score: 0.8747 - val_loss: 0.2716 - learning_rate: 6.1126e-05
Epoch 5/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 336s 529ms/step - iou_score: 0.8712 - loss: 0.2736 - val_iou_score: 0.8782 - val_loss: 0.2655 - learning_rate: 3.8874e-05
Epoch 6/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 336s 529ms/step - iou_score: 0.8723 - loss: 0.2699 - val_iou_score: 0.8789 - val_loss: 0.2622 - learning_rate: 1.8826e-05
Epoch 7/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 335s 

2026-02-08 16:18:14.718566: E external/local_xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module a_inference_one_step_on_data_1461080__.89303] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2026-02-08 16:18:35.593424: E external/local_xla/xla/service/slow_operation_alarm.cc:140] The operation took 2m20.875045676s

********************************
[Compiling module a_inference_one_step_on_data_1461080__.89303] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


635/635 ━━━━━━━━━━━━━━━━━━━━ 641s 568ms/step - iou_score: 0.7572 - loss: 0.5823 - val_iou_score: 0.8580 - val_loss: 0.3206 - learning_rate: 1.0000e-04
Epoch 2/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 340s 535ms/step - iou_score: 0.8539 - loss: 0.3176 - val_iou_score: 0.8554 - val_loss: 0.3210 - learning_rate: 9.5048e-05
Epoch 3/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 343s 541ms/step - iou_score: 0.8653 - loss: 0.2893 - val_iou_score: 0.8650 - val_loss: 0.2972 - learning_rate: 8.1174e-05
Epoch 4/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 343s 540ms/step - iou_score: 0.8690 - loss: 0.2782 - val_iou_score: 0.8751 - val_loss: 0.2746 - learning_rate: 6.1126e-05
Epoch 5/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 342s 538ms/step - iou_score: 0.8749 - loss: 0.2661 - val_iou_score: 0.8752 - val_loss: 0.2714 - learning_rate: 3.8874e-05
Epoch 6/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 341s 537ms/step - iou_score: 0.8737 - loss: 0.2660 - val_iou_score: 0.8782 - val_loss: 0.2648 - learning_rate: 1.8826e-05
Epoch 7/7
635/635 ━━━━━━━━━━━━━━━━━━━━ 340s 

In [10]:
import os
import cv2
import glob
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import segmentation_models as sm
from tqdm.notebook import tqdm

# --- 1. CONFIGURATION ---
DATASET_ROOT = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle"
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "test_images_padded")

# 🟢 CHANGE YOUR THRESHOLD HERE 🟢
NEW_THRESHOLD = 0.5  # <--- Edit this value (e.g. 0.3, 0.45, 0.6)

# Model Settings (Must match training)
IMG_H = 544 
IMG_W = 960 
BACKBONE = 'seresnext50'
preprocess_input = sm.get_preprocessing(BACKBONE)
N_FOLDS = 5

# --- 2. HELPER FUNCTIONS ---
def build_model():
    # Re-construct architecture to load weights into
    return sm.Unet(
        BACKBONE, 
        classes=1, 
        activation='sigmoid', 
        encoder_weights=None, 
        decoder_block_type='transpose'
    )

def rle_encode(mask):
    pixels = mask.flatten(order="F")
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# --- 3. INFERENCE LOOP ---
test_files = sorted(glob.glob(os.path.join(TEST_IMG_DIR, "*.png")))
accumulated_preds = np.zeros((len(test_files), IMG_H, IMG_W, 1), dtype=np.float16)

print(f"📝 Generatng CSV with Threshold: {NEW_THRESHOLD}")
print(f"📂 Processing {len(test_files)} images...")

for fold in range(1, N_FOLDS + 1):
    model_path = f'model_fold_{fold}.keras'
    
    if not os.path.exists(model_path):
        print(f"⚠️ Warning: {model_path} not found. Skipping.")
        continue
        
    print(f"➡️ Loading Fold {fold}...")
    
    # Clear Memory
    tf.keras.backend.clear_session()
    model = build_model()
    model.load_weights(model_path)
    
    # Predict in batches
    batch_size = 4
    for i in tqdm(range(0, len(test_files), batch_size), desc=f"Fold {fold}"):
        batch_files = test_files[i : i + batch_size]
        batch_imgs = []
        
        for path in batch_files:
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_W, IMG_H))
            img = preprocess_input(img)
            batch_imgs.append(img)
            
        batch_imgs = np.array(batch_imgs)
        
        # 1. Prediction (Standard)
        p1 = model.predict(batch_imgs, verbose=0)
        
        # 2. Prediction (Flip TTA)
        p2 = model.predict(np.fliplr(batch_imgs), verbose=0)
        p2 = np.fliplr(p2)
        
        # Average
        batch_pred = (p1 + p2) / 2.0
        accumulated_preds[i : i + batch_size] += batch_pred.astype(np.float16)
    
    del model
    gc.collect()

# Average across folds
accumulated_preds /= N_FOLDS

# --- 4. APPLY THRESHOLD & SAVE ---
print(f"💾 Saving to submission.csv with threshold {NEW_THRESHOLD}...")
submission_rows = []

for i, path in enumerate(tqdm(test_files, desc="Encoding")):
    # Resize to original 540x960
    pred_mask = accumulated_preds[i].astype(np.float32)
    prob_full = cv2.resize(pred_mask, (960, 540))
    
    # 🟢 APPLY CUSTOM THRESHOLD HERE 🟢
    mask = (prob_full > NEW_THRESHOLD).astype(np.uint8)
    
    # Optional: Fill small holes (Morphology)
    # kernel = np.ones((5, 5), np.uint8)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) 
    
    rle = rle_encode(mask)
    name = os.path.splitext(os.path.basename(path))[0]
    submission_rows.append({'image_id': name, 'encoded_pixels': rle})

df_sub = pd.DataFrame(submission_rows)
df_sub.to_csv("submission.csv", index=False)
print("✅ Done!")

📝 Generatng CSV with Threshold: 0.45
📂 Processing 1002 images...
➡️ Loading Fold 1...


Fold 1:   0%|          | 0/251 [00:00<?, ?it/s]

➡️ Loading Fold 2...


Fold 2:   0%|          | 0/251 [00:00<?, ?it/s]

➡️ Loading Fold 3...


...

➡️ Loading Fold 5...


Fold 5:   0%|          | 0/251 [00:00<?, ?it/s]

💾 Saving to submission.csv with threshold 0.45...


Encoding:   0%|          | 0/1002 [00:00<?, ?it/s]

✅ Done!
